In [12]:
from bs4 import BeautifulSoup
import urllib.request
import requests
from pprint import pprint
import pandas as pd
import json
import copy
from datetime import datetime, timezone
from google.cloud import storage
import gcsfs
import re
from typing import List

In [13]:
storage_client = storage.Client(project="ytone-430507")
today = datetime.now()
table_name = "douyin_hotcomment"
api_name = "get_author_hot_comment_tokens"

In [14]:
hotcomment_data = []
hotcomment_columns = ["rate", "content", "influencer_id"]

In [15]:
processing_blobs = [
{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[1], "%Y-%m-%d"),
    "batch_number": int(blob.name.split('/')[-1].replace(".json", "").split("_")[-1]),
    "scrape_date": blob.name.split('/')[-1].replace(".json", "").split("_")[-2]
} for blob in storage_client.list_blobs("0_raw_data",prefix="2_xingtu/") if api_name in blob.name]
processing_blobs

[{'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-11/get_author_hot_comment_tokens_0911_0.json, 1726037772845714>,
  'date': datetime.datetime(2024, 9, 11, 0, 0),
  'batch_number': 0,
  'scrape_date': '0911'},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-13/get_author_hot_comment_tokens_0913_0.json, 1726209878966367>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': 0,
  'scrape_date': '0913'},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-13/get_author_hot_comment_tokens_0913_1.json, 1726209879547030>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': 1,
  'scrape_date': '0913'},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-13/get_author_hot_comment_tokens_0913_2.json, 1726209880155079>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': 2,
  'scrape_date': '0913'},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-13/get_author_hot_comment_tokens_0913_3.json, 1726209880743108>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number':

In [16]:
processed_blobs = [{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[2], "%Y-%m-%d"),
    "batch_number": int(blob.name.split('/')[-1].replace(".parquet", "").split("_")[-1]),
    "scrape_date": int(blob.name.split('/')[-1].replace(".parquet", "").split("_")[-2]),
} for blob in storage_client.list_blobs("4_data_warehouse",prefix="1_xingtu/") if table_name == blob.name.split("/")[1] and len(blob.name.split("/")) == 4]
processed_blobs

[{'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_hotcomment/2024-09-08/douyin_hotcomment_240917_0.parquet, 1726568052386224>,
  'date': datetime.datetime(2024, 9, 8, 0, 0),
  'batch_number': 0,
  'scrape_date': 240917},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_hotcomment/2024-09-08/douyin_hotcomment_240917_1.parquet, 1726568151807526>,
  'date': datetime.datetime(2024, 9, 8, 0, 0),
  'batch_number': 1,
  'scrape_date': 240917},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_hotcomment/2024-09-09/douyin_hotcomment_240917_0.parquet, 1726550714121472>,
  'date': datetime.datetime(2024, 9, 9, 0, 0),
  'batch_number': 0,
  'scrape_date': 240917},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_hotcomment/2024-09-11/douyin_hotcomment_240917_0.parquet, 1726550690740629>,
  'date': datetime.datetime(2024, 9, 11, 0, 0),
  'batch_number': 0,
  'scrape_date': 240917},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_hotcomment/2024-09-13/douyin_hotcomment_240917_0.parquet, 17

In [17]:
to_process = []
for processing_blob in processing_blobs:
    if processing_blob["date"] >= datetime(2024, 9, 13):
        if processing_blob["batch_number"] not in [processed_blob["batch_number"] for processed_blob in processed_blobs if processing_blob["date"] == processed_blob["date"]]:
            to_process.append(processing_blob)
pprint(to_process)
print(len(to_process))

[{'batch_number': 0,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_author_hot_comment_tokens_1011_0.json, 1728638388172339>,
  'date': datetime.datetime(2024, 10, 11, 0, 0),
  'scrape_date': '1011'},
 {'batch_number': 1,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_author_hot_comment_tokens_1011_1.json, 1728638389228881>,
  'date': datetime.datetime(2024, 10, 11, 0, 0),
  'scrape_date': '1011'}]
2


In [18]:
def get_douyin_hot_comment_token_data(raw_data):
    result = []
    for data in raw_data:
        record = {}
        get_author_hot_comment_tokens = data.get("get_author_hot_comment_tokens")
        hot_comment_tokens = get_author_hot_comment_tokens.get("hot_comment_tokens")
        if hot_comment_tokens:
            for hot_comment_token in hot_comment_tokens:
                record = {}
                record["influencer_id"] = data["profile_id"]
                record["content"] = hot_comment_token["comment_token"]
                record["rate"] = hot_comment_token["hot_rate"]
                result.append(record)
    return result


In [19]:
for item in to_process:
    raw_data = json.loads(item["blob"].download_as_string())
    hotcomment_data = get_douyin_hot_comment_token_data(raw_data)
    pd.DataFrame(hotcomment_data, columns=hotcomment_columns).to_parquet("gs://4_data_warehouse/1_xingtu/douyin_hotcomment/" + str(item["date"].date()) + "/douyin_hotcomment_" + item["scrape_date"] + "_" + str(item["batch_number"]) + ".parquet")